In [10]:
import mysql.connector as msql
from mysql.connector import Error
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from pyproj import CRS
from shapely.geometry import Point
import movingpandas as mpd
from datetime import datetime, timedelta
import math
import numpy as np
from scipy.stats import entropy
from scipy.stats import norm, kurtosis
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

In [11]:
def shannon_entropy(hist):
    side, count = np.unique(hist, return_counts=True)
    dist = count/len(hist)
    entropy_value = entropy(dist)
    return entropy_value
  
def kurtosis_fisher(dist):
    return kurtosis(dist)

def kurtosis_pearson(dist):
    return kurtosis(dist, fisher=False)

In [12]:
def stop_points_detection(uids):
    # unique trajectories
    count = 1
    total = len(uids)
    
    for uid in uids:      

        uid = uid[0]
        feature_query = "SELECT %s FROM trajectories.gpspoints_sample WHERE identifier = '%s';" % ('timestamp, latitude, \
                                                                                            longitude, altitude, \
                                                                                            transport_mode, identifier',
                                                                                            uid)        
        mycursor.execute(feature_query)
        data = mycursor.fetchall()
        
        if(len(data) >= 80):    
            df = pd.DataFrame(data, columns = ['timestamp', 'latitude', 'longitude', 'altitude', 'transport_mode', 'identifier'])
            df['geometry'] = [Point(long, lat) for long,lat in zip(df['longitude'].to_list(), df['latitude'].to_list())]             
            geodata = gpd.GeoDataFrame(df, crs = CRS.from_epsg('4326')).set_index('timestamp')
            traj_collection = mpd.TrajectoryCollection(geodata, 'transport_mode')

            Hours = .005
            SearchRadio = 200
            stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(hours=Hours), 
                                                                                  max_diameter=SearchRadio)

            duration = []
            length = []
            label = df['transport_mode'].unique()[0]
            
            for stoptrack in stops.trajectories:
                # add stop duration in hours
                duration.append(stoptrack.get_duration().total_seconds())
                # add length
                length.append(stoptrack.get_length())

            row = (uid, max(duration), np.mean(duration), min(duration), shannon_entropy(duration), 
                   kurtosis_fisher(duration), kurtosis_pearson(duration),
                  max(length), np.mean(length), min(length), shannon_entropy(length), 
                   kurtosis_fisher(length), kurtosis_pearson(length), label)
            
            feature_query = '''INSERT INTO trajectories.stop_point_features_sample VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s);'''        
            mycursor.execute(feature_query, row)
        
            count += 1
        

In [13]:
mydb = msql.connect(
  host="localhost",
  user="root",
  password="root",
  database="trajectories"
)
mycursor = mydb.cursor(buffered=True)

try:
    if mydb.is_connected():

        uids_query = """SELECT DISTINCT identifier FROM trajectories.gpspoints_sample;"""
        mycursor.execute(uids_query)
        uids = mycursor.fetchmany(1000)
        stop_points_detection(uids)
        mydb.commit()

except Error as e:

    print("Error while connecting to gpspoints:", e)

finally:
    if mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("MySQL connection is closed")

C:\ProgramData\Miniconda3\lib\site-packages\geopy\point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


MySQL connection is closed


ValueError: Latitude must be in the [-90; 90] range.